# Currency Crisis Data Visuals

In [ ]:
%load_ext watermark
%watermark

import numpy as np
import requests  # Python 3.6
import pandas as pd           # pandas version 0.22
import seaborn as sns
import datetime as dt

from ipywidgets import widgets
from IPython.display import display, clear_output, HTML

%matplotlib inline

import hvplot.pandas
import holoviews as hv
from holoviews import opts


hv.extension('bokeh', 'matplotlib')


%watermark -p pandas,numpy,matplotlib,seaborn,holoviews,hvplot

# Variables, Functions, Class and other Settings for further Data Analysis

### Getting Data from International Monetary Fund Data Services

In [ ]:
# Class to operate with IMF databases and dataseries
 
# http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{database ID}/{frequency}.{item1 from
# dimension1}+{item2 from dimension1}+{item N from dimension1}.{item1 from
# dimension2}+{item2 from dimension2}+{item M from dimension2}?startPeriod={start
# date}&endPeriod={end date}

# example of access link:
# http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/BOP/Q.KZ.BK_DB_BP6_USD?startPeriod=2015-01-01&endPeriod=2018-12-31
         

class Data_IMF():
    """This class to interect with IMF JSON API access for downloading datseries
    """
    # JSON API access link
    url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
    method = ['Dataflow', 'Datastructure', 'CompactData']

    def __init__(self, db='IFS', freq='Q', country='KZ', ind='NGDP_XDC', start='2005-01-01', end='2018-12-31'):
        """Method to initiate object and download series of data
        """
        self.key = f"{db}/{freq}.{country}.{ind}"
        self.period = f"?startPeriod={start}&endPeriod={end}"
        self.freq = freq
        self.start = pd.to_datetime(start)
        self.end = pd.to_datetime(end)
        self.series = self._get_data() #pd.DataFrame
         
    def _get_data(self):
        """The method to download JSON dataseries from IMF and convert them into pandas series
        """
        url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
        key = 'CompactData/' + self.key + self.period
        try:
            print('Downloading data from ', self.key)
            r = requests.get(f'{url}{key}').json()
        except:
            print('Something goes wrong')
            return None
            
        data = pd.DataFrame()
        try:
            obs = r['CompactData']['DataSet']['Series']['Obs']
            data = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
                float(i['@OBS_VALUE']) 
                for i in obs})
            if self.freq=='Q':
                # Adjust quarter data to coorespond the end of the quarter
                data.index = data.index + pd.offsets.QuarterEnd()
            elif self.freq=='M':
                # Adjust quarter data to coorespond the end of the month
                data.index = data.index + pd.offsets.MonthEnd()
                
        except:
            print('Smth goes wrong')
            
        self.series = data
        return data
        
    def find_imf_data(self):
        # TBD
        return None


In [ ]:
# Framework
start = pd.to_datetime('2010-01-01')
end = pd.to_datetime('2018-01-01')
finish = end

### Visuals

In [ ]:
# visual decoration

cycle_phases = hv.VLine(pd.to_datetime('2010-01-01')).opts(line_width=1, color='orange') *\
               hv.VLine(pd.to_datetime('2014-01-01')).opts(line_width=1, color='orange') *\
               hv.VLine(pd.to_datetime('2016-12-31')).opts(line_width=1, color='orange') *\
               hv.VLine(pd.to_datetime('2018-12-31')).opts(line_width=1, color='orange')
phases = cycle_phases

def arrow(series, date=pd.to_datetime('2014-02-23'), label=""):
    x = series.index[series.index.get_loc(date, method='nearest')]
    try:
        y = series.loc[x].values[0]
    except:
        y = series.loc[x]
    return hv.Arrow(date, y, label, 'v', arrowstyle='->')

In [ ]:
opts.defaults(
    #opts.Scatter(width=900, height=450, xlim=(start, finish)),
    #opts.Bars(width=900, height=450, xlim=(start, finish)),
    opts.Curve(width=900, height=250, padding=(0, (0, 0.5)), xlim=(start, end)) # padding upper y bound
)


# Dataseries Visuals
**Groups of figures:**
- Main
- Indebtedness
- Monetary and Fiscal Policy
- Economic conditions
- Markets
- External Position

**The Phases of the Classic Inflationary Debt Cycle**
1. The Early Part of the Cycle
2. The Bubble
3. The Top and Currency Defense
4. The Depression (Often When the Currency Is Let Go)
5. Normalization

## Local Currency Tenge Exchange Rate to USD
Source: https://nationalbank.kz

In [ ]:
tenge_to_currencies = pd.read_excel('tenge_data/Official_Foreign_Exchange_Rates_NBRK_on_11_02_2019(2).xls',
                                    index_col=0, parse_dates=True)

In [ ]:
tenge_to_usd = tenge_to_currencies['USD']
tenge_to_usd = tenge_to_usd[tenge_to_usd.astype(bool)]
tenge_to_usd.tail()

In [ ]:
tenge_to_usd = pd.DataFrame(tenge_to_usd)
tenge_to_usd['USDpct'] = tenge_to_usd.pct_change()

In [ ]:
tenge_to_usd = tenge_to_usd.loc[start:end]

In [ ]:
#tenge_to_usd['USDpct']
crisis09 = tenge_to_usd.loc[(tenge_to_usd.index.year == 2009), 'USDpct'].idxmax()
crisis14 = tenge_to_usd.loc[(tenge_to_usd.index.year == 2014), 'USDpct'].idxmax()
crisis15 = tenge_to_usd.loc[(tenge_to_usd.index.year == 2015), 'USDpct'].idxmax()
print(crisis09, crisis14, crisis15)

In [ ]:
#%%opts Curve [width=900, height=250]

currency_plot1 = hv.Curve(tenge_to_usd.reset_index(), ('Date',' '), ('USD', 'Tenge'))\
    .options(title_format="Tenge to USD Exchange Rate") *\
    arrow(tenge_to_usd['USD'], crisis14, label='Currency\ncrisis 14') *\
    arrow(tenge_to_usd['USD'], crisis15, label='Currency\ncrisis 15')

currency_plot2 = hv.Curve(tenge_to_usd.reset_index(), 'Date', ('USDpct', '%'))\
    .options(title_format="Tenge to USD Exchange Rate Daily Change") *\
     cycle_phases * hv.Arrow((crisis14+dt.timedelta(weeks=5)), tenge_to_usd.loc[crisis14, 'USDpct'],
                             f"{round(tenge_to_usd.loc[crisis14, 'USDpct']*100,1)}% fall", 'v', arrowstyle='-[')   *\
                    hv.Arrow((crisis15+dt.timedelta(weeks=5)), tenge_to_usd.loc[crisis15, 'USDpct'],
                             f"{round(tenge_to_usd.loc[crisis15, 'USDpct']*100,1)}% fall", 'v', arrowstyle='-[')

In [ ]:
currency_plot = currency_plot1 + currency_plot2
currency_plot.cols(1)

In [ ]:
#hv.save(currency_plot, 'currency_plot.png')

## Brend (USD per barrel)

Europe Brent Spot Price FOB (Dollars per Barrel)
https://www.eia.gov/dnav/pet/hist_xls/RBRTEd.xls

NYMEX Futures Prices
https://www.eia.gov/dnav/pet/xls/PET_PRI_FUT_S1_D.xls

In [ ]:
brent_spot = pd.read_excel('https://www.eia.gov/dnav/pet/hist_xls/RBRTEd.xls',
                           skiprows=2, sheet_name="Data 1", index_col=0, parse_dates=True)

In [ ]:
wti_spot = pd.read_excel('https://www.eia.gov/dnav/pet/hist_xls/RWTCd.xls',
                              skiprows=2, sheet_name="Data 1", index_col=0, parse_dates=True)

In [ ]:
wti_futures = pd.read_excel('https://www.eia.gov/dnav/pet/xls/PET_PRI_FUT_S1_D.xls',
                              skiprows=2, sheet_name="Data 1", index_col=0, parse_dates=True)

In [ ]:
wti_futures[wti_futures.index.year==2014].join(wti_spot[wti_spot.index.year==2014]).join(brent_spot[brent_spot.index.year==2014]).plot(figsize=(15,10));

In [ ]:
(currency_plot + phases * brent_spot.hvplot(legend='top_left', width=900, height=350)).cols(1)

In [ ]:
brent.head(10)

In [ ]:
#%%opts Curve [width=900, height=250]
brent_plot = hv.Curve(brent_spot.reset_index(), 'Date',
                         'Europe Brent Spot Price FOB (Dollars per Barrel)', label='Brent').\
                    options(title_format="Europe Brent Spot Price FOB (Dollars per Barrel)").opts(xlim=(start, finish),
                                                                ylabel='USD',
                                                               padding=(0, (0, 0.5))) * phases *\
    arrow(brent_spot, label='Currency\ncrisis 14') *\
    arrow(brent_spot, date=crisis15, label='Currency\ncrisis 15') * \
    arrow(brent_spot, date=crisis09, label='Currency\ncrisis 09')

brent_plot

In [ ]:
(currency_plot + brent_plot).cols(1)

futures

In [ ]:
wti = pd.DataFrame(wti_spot).join(wti_futures)

In [ ]:
wti_plot = wti.hvplot(xlim=(pd.to_datetime('2013-06-01'), pd.to_datetime('2014-06-01')), ylim=(90,130),
          legend='top_left', width=900, height=400, alpha=0.7, ylabel='USD per Barrel',
           title='Spot Price mostly exceeds Futures Price of WTI from 06/01/2013 till 06/01/2014') *\
    arrow(wti_spot, label='Currency\ncrisis 14')* phases +\
    wti.hvplot(xlim=(pd.to_datetime('2012-06-01'), pd.to_datetime('2013-06-01')), ylim=(75,100),
          legend=False, width=900, height=250, alpha=0.7, ylabel='USD per Barrel',
          title='Futures Price mostly exceeds Spot Price of WTI from 06/01/2012 till 06/01/2013',
              shared_axes=False)

wti_plot.cols(1)

In [ ]:
wti[wti.index>pd.to_datetime('2018-09-01')].plot(figsize=(15,10), grid=True);

In [ ]:
#hv.save(wti_plot, 'wti_plot.png')

## DGP (local currency, USD, real indexed)
Source: nationalbank.kz`

In [ ]:
# newer version...

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/Q.KZ.NGDP_XDC' # adjust codes here
r = requests.get(f'{url}{key}').json()
obs = r['CompactData']['DataSet']['Series']['Obs']
gdp = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
            float(i['@OBS_VALUE']) 
            for i in obs})

In [ ]:
gdp = Data_IMF()

In [ ]:
gdp.series.reset_index().head() #.columns

In [ ]:
%%opts Curve [width=900, height=250]
gdp_plot = hv.Curve(gdp.series.reset_index(), 'index',
                         '0', label='Brent').\
                    options(title_format="GDP (bln. USD)").opts(
                                                                ylabel='bln. USD',
                                                               padding=(0, (0, 0.5))) * phases # +\
gdp_plot

In [ ]:
    hv.Curve(gdp.series.pct_change().reset_index(), 'index',
                         '0').\
                    options(title_format="GDP % Chg", shared_axes=False).opts(ylabel='%',
                                                               padding=(0, (0, 0.5))) * phases 

gdp_plot.cols(1)

In [ ]:
gdp[gdp.index.year==2013]

In [ ]:
gdp.reset_index().to_csv('KZ_gdp_Q.csv')

In [ ]:
gdp_plot.cols(1)

In [ ]:
%%opts Curve [width=900, height=250]
gdp_plot = hv.Curve(debts1.iloc[:,18].reset_index(), 'index',
                         '  GDP for the 12 months (bln. USD)***', label='Brent').\
                    options(title_format="GDP (bln. USD)").opts(xlim=(start, finish),
                                                                ylabel='bln. USD',
                                                               padding=(0, (0, 0.5))) * phases +\
    hv.Curve(debts1.iloc[:,18].pct_change().reset_index(), 'index',
                         '  GDP for the 12 months (bln. USD)***').\
                    options(title_format="GDP % Chg", shared_axes=False).opts(xlim=(start, finish), ylim=(-10,0),
                                                                ylabel='%',
                                                               padding=(0, (0, 0.5))) * phases 

gdp_plot.cols(1)

## Real Growth (Y/Y)

## Total Debt (%GDP) 
and debt service
Source: IMF

In [ ]:
debts = pd.read_excel('nbk_data/ED_indicators_eng_3-18_доп.xls', skiprows=2, index_col=0)

In [ ]:
debts.head()

col_quarters = [not str(i).isnumeric() for i in debts.columns]

col_quart_names = debts.columns[col_quarters] #.reset_index().apply(lambda x: x)

debts1 = debts[col_quart_names]

debts1.columns = pd.to_datetime(col_quart_names.map(lambda x: x[-4:] + '-' + str(int(x[0])*3-2)+'-'+'1'), format="%Y/%m/%d")

debts1 = debts1.T.dropna(axis=1, how='all')

debts2 = debts1.iloc[:, [0,3]].div(debts1.iloc[:, 18], axis=0).div(1000).mul(100)

In [ ]:
debts2 = debts2[((debts2.index> start) & (debts2.index < finish))]

In [ ]:
%%opts Curve [width=900, height=250]
debts_plot = hv.Curve(debts2.reset_index(), 'index',
                         '1. External debt (estimate at the end of period)', label='External Debt').\
                    options(title_format="Debts to GDP").opts(xlim=(start, finish), ylabel='percent') * phases +\
    hv.Curve(debts2.reset_index(), 'index',
                         '3. Long-term external debt service (incl. intercompany lending)')\
    .opts(xlim=(start, finish), ylabel='percent') * phases

In [ ]:
%%opts Curve [width=900, height=350]
%%opts Bars [width=900, height=250]
debts_plot1 = hv.Curve(debts2.reset_index(), 'index',
                         '1. External debt (estimate at the end of period)', label='External Debt').\
                    options(title_format="Debts to GDP").opts(xlim=(start, finish), ylabel='%',
                                                             ylim=(0,140)) *\
        arrow(debts2['1. External debt (estimate at the end of period)'], label='Currency\ncrisis 14') *\
    arrow(debts2['1. External debt (estimate at the end of period)'], date=crisis15, label='Currency\ncrisis 15')

debts_plot2 = hv.Bars(debts2.pct_change().reset_index(), 'index',
                         '1. External debt (estimate at the end of period)', label='External Debt').\
                    options(title_format="Debts to GDP percent change").opts(xlim=(start, finish), ylabel='%',
                                                             shared_axes=False,xaxis=None)

In [ ]:
debts_plot = (debts_plot1 + debts_plot2).cols(1)
debts_plot
#.plot(secondary_y=['3. Long-term external debt service (incl. intercompany lending)'])

In [ ]:
hv.save(debts_plot, 'debts_plot.png')

## National Fund of Kazakhstan

In [ ]:
reserves_funds = pd.read_excel('nbk_data/stat.xlsx', skiprows=4, skipfooter=3) #, dtype='str')

In [ ]:
reserves_funds.tail()

In [ ]:
reserves_funds.iloc[:,0] = reserves_funds.iloc[:,0].str.strip(' \t\r\n')

In [ ]:
reserves_funds.index = pd.to_datetime(reserves_funds.iloc[:,0],
                        format='%m.%y', errors='coerce', exact=False) + pd.offsets.MonthEnd()

In [ ]:
national_oil_fund = reserves_funds.iloc[:, [-3].div(1000).dropna() #.hvplot()

In [ ]:
reserves_nbk = pd.DataFrame({'Gross International Reserves': pd.to_numeric(reserves_funds.iloc[:,1].str.replace(' ','')),
              'National Oil Fund': reserves_funds.iloc[:, -3]}).dropna().div(1000)
#.strip(" ")

In [ ]:
reserves_plot = reserves_nbk.hvplot.area(stacked=True, 
                                         xlim=(pd.to_datetime('2010-01-01'),
                                               pd.to_datetime('2018-12-31')),
                                         ylim=(0,140),
          legend='top_left', width=900, height=400, alpha=0.9,
           xlabel='Date', ylabel='bln. USD', title='Reserves of Kazakhstan') *\
    arrow(reserves_nbk.sum(axis=1), label='Currency\ncrisis 14') *\
    arrow(reserves_nbk.sum(axis=1), date=crisis15, label='Currency\ncrisis 15')
reserves_change = reserves_nbk.sum(axis=1).pct_change().hvplot.bar(shared_axes=False) #* hv.HLine(0)

In [ ]:
(reserves_plot) # + reserves_change).cols(1)

In [ ]:
hv.save(reserves_plot, 'reserves_plot.png')

## Current Account (%GDP)
Sourse: IMF (BOP, ...)
IFS/Q.KZ.BG_BP6_USD

In [ ]:
gdp = Data_IMF(db='IFS', ind='NGDP_XDC')

In [ ]:
gdp.series.head()

In [ ]:
new = debts1.iloc[:, 18]
new.index = new.index + pd.offsets.QuarterEnd()

In [ ]:
cur_acc1 = pd.concat([gdp.series, new], axis=1)

In [ ]:
cur_acc1.head()

In [ ]:
cur_acc1.columns = ['Current Account', 'GDP']

In [ ]:
cur_acc1_pc = cur_acc1['Current Account'].div(cur_acc1['GDP']*1e3).mul(100).reset_index()

In [ ]:
cur_acc1_pc.columns = ['Date', 'Current Account (% GDP)']

In [ ]:
cur_acc_plot = hv.Curve(cur_acc1_pc, 'Date', 'Current Account (% GDP)', label='Current Account').\
                    options(title_format="Current Account (% GDP)").opts(xlim=(start, finish), ylabel='%') * phases
cur_acc_plot #.cols(1)

In [ ]:
cur_acc_plot

## GDP Gap??

## Real FX vs TWI??

## Capital Inflows and Outwlows
%GDP
(%PGDP)

In [ ]:
cap_cd = Data_IMF(db='BOP', ind='BK_CD_BP6_USD')

In [ ]:
cap_db = Data_IMF(db='BOP', ind='BK_DB_BP6_USD')

In [ ]:
capital = pd.DataFrame({'Credit': cap_cd.series, 'Debit': cap_db.series})

In [ ]:
capital.head()

In [ ]:
%%opts Curve [width=900, height=250]
capital_plot = hv.Curve(capital.reset_index(), 'index',
                         'Credit', label='Credit').\
                    options(title_format="Capital Inflow").opts(xlim=(start, finish),
                                                                ylabel='USD',
                                                               padding=(0, (0, 0.5))) * phases *\
    arrow(capital['Credit'], label='Currency\ncrisis 14') *\
    arrow(capital['Credit'], date=crisis15, label='Currency\ncrisis 15') * \
    arrow(capital['Credit'], date=crisis09, label='Currency\ncrisis 09') + \
    hv.Curve(capital.reset_index(), 'index',
                         'Debit', label='Debit').\
                    options(title_format="Capital Outflow").opts(xlim=(start, finish),
                                                                ylabel='USD',
                                                               padding=(0, (0, 0))) * phases *\
    arrow(capital['Debit'], date=crisis15, label='Currency\ncrisis 15') * \
    arrow(capital['Debit'], date=crisis14, label='Currency\ncrisis 14')

capital_plot.cols(1)

In [ ]:
capital_plot1 = capital.hvplot(title="Capital Inflow / Outflow", ylabel='bln. USD',
                               xlim=(pd.to_datetime('2012-01-01'),
                                    pd.to_datetime('2018.01.01')),
                              ylim=(0,270))  *\
    arrow(capital['Credit'], label='Currency\ncrisis 14') *\
    arrow(capital['Credit'], date=crisis15, label='Currency\ncrisis 15')
    
capital_plot1

In [ ]:
hv.save(capital_plot1, 'capital_flow_plot.png')

## Growth vs Potential????

## Nominal Short Rate???
Source: nationalbank.kz, IMF

refinance rate
Financial, Interest Rates, Refinancing Rate, Percent per annum, FIR_PA


Base interest rate of the NBK
278	Financial, Interest Rates, Monetary Policy-Related Interest Rate, Percent per annum	FPOLM_PA




In [ ]:
ref_rate = Data_IMF(freq='M', ind='FIR_PA')

In [ ]:
inter_rate = Data_IMF(freq='M', ind='FPOLM_PA')

In [ ]:
rates = pd.concat([ref_rate.series, inter_rate.series], axis=1)

In [ ]:
rates.columns = ['Refinance Rate', 'Base Interest Rate of NBK']

In [ ]:
rates_plot = rates.hvplot(title="Iterest Rates", ylabel='%', xlabel='Date', ylim=(0,20),
                         legend='top_left', width=900)  *\
    arrow(rates['Refinance Rate'], label='Currency\ncrisis 14') *\
    arrow(rates['Refinance Rate'], date=crisis15, label='Currency\ncrisis 15')
    
rates_plot

In [ ]:
hv.save(rates_plot, 'rates_plot.png')

## Reserves (USD)
Source: IMF, IFS/RAFA_USD

In [ ]:
#  Reserves

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/Q.KZ.RAFA_MV_USD' # adjust codes here
r = requests.get(f'{url}{key}').json()
obs = r['CompactData']['DataSet']['Series']['Obs']
reserves = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
            float(i['@OBS_VALUE']) 
            for i in obs})

In [ ]:
reserves.tail(10).div(1e3).round(1)

In [ ]:
# Total Reserves

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/Q.KZ.RAFA_USD' # adjust codes here
r = requests.get(f'{url}{key}').json()
obs = r['CompactData']['DataSet']['Series']['Obs']
reserves1 = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
            float(i['@OBS_VALUE']) 
            for i in obs})

In [ ]:
reserves1.tail(10).div(1e3).round(1)

In [ ]:
reserves1.to_csv('total_reserves.csv')

In [ ]:
reserves = pd.read_csv('total_reserves.csv', index_col=0, parse_dates=True, names=['Reserves'])

In [ ]:
reserves.iloc[1:] = reserves.iloc[:-1].values

In [ ]:
reserves.reset_index().tail()

In [ ]:
reserves.reset_index().tail()

In [ ]:
%%opts Curve [width=900, height=250]
reserves_plot = hv.Curve(reserves.div(1e3).reset_index(), 'index',
                         'Reserves', label='Reserves').\
                    options(title_format="Reserves (USD)").opts(xlim=(start, finish),
                                                                ylabel='bln. USD',
                                                               padding=(0, (0, 0.5))) * phases

reserves_plot #.cols(1)

## Equity Price (Indexed)
Source: KASE

In [ ]:
kase = pd.read_excel('kase_data/Index_KASE_190302.xls', skiprows=1, index_col=0)

In [ ]:
kase.index = pd.to_datetime(kase.index, format='%d.%m.%y')

In [ ]:
kase.index[0]

In [ ]:
kase_plot = hv.Curve(kase.reset_index(), 'Date', 'Close').opts(title='KASE Index Close Price') *\
    arrow(kase['Close'], label='Currency\ncrisis 14') *\
    arrow(kase['Close'], date=crisis15, label='Currency\ncrisis 15') +\
  hv.Curve(kase.reset_index(), 'Date', 'Volume, USD th').opts(shared_axes=False, logy=True, title='KASE Volume')

kase_plot.cols(1)

In [ ]:
hv.save(kase_plot, 'kase_plot.png')

### TONIA

In [ ]:
tonia = pd.read_excel('kase_data/TONIA_190302.xls', skiprows=1, index_col=0)

In [ ]:
tonia.index = pd.to_datetime(tonia.index, format='%d.%m.%y')

In [ ]:
tonia.index[0]

In [ ]:
tonia.columns

In [ ]:
tonia_plot = hv.Curve(tonia.reset_index(), 'Date', 'mm_indi_table_close').opts(title='TONIA Index Close Price',
                                                                              ylabel='TONIA', ylim=(0,200)) *\
    arrow(tonia['mm_indi_table_close'], label='Currency\ncrisis 14') *\
    arrow(tonia['mm_indi_table_close'], date=crisis15, label='Currency\ncrisis 15') +\
  hv.Curve(tonia.reset_index(), 'Date', 'Transactions amount, m USD').opts(shared_axes=False, logy=False, title='TONIA Volume')

tonia_plot.cols(1)

In [ ]:
hv.save(tonia_plot, 'tonia_plot.png')

## FX Debt (%GDP)

## Core Inflatoin
(Y/Y) %
abs

# Core Inflation

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.KZ.PCPI_PC_CP_A_PT' # adjust codes here

key_ = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/BOP/Q.KZ.BXGS_BP6_USD+BMGS_BP6_USD?startPeriod=2015-01-01&endPeriod=2018-12-31'
key_ = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/M.KZ.PCPI_PC_CP_A_PT?startPeriod=2015-01-01&endPeriod=2018-12-31'

abc = requests.get(key_)
r = abc.json()

In [ ]:
inflation = Data_IMF(db='IFS', freq='M', ind='PCPI_PC_CP_A_PT')

In [ ]:
inflation.series.name = 'CPI'

In [ ]:
inflation.series.reset_index().columns

inflation.series.index = inflation.series.index + pd.offsets.MonthEnd()

In [ ]:
%%opts Curve [width=900, height=250]
inflatin_plot = hv.Curve(inflation.series.reset_index(), 'index',
                         'CPI', label='Inflation').\
                    opts(title_format="Inflation CPI Y/Y", xlabel='Date', ylabel='%', ylim=(0,20)) *\
    arrow(inflation.series, label='Currency\ncrisis 14') *\
    arrow(inflation.series, date=crisis15, label='Currency\ncrisis 15')
inflatin_plot

In [ ]:
hv.save(inflatin_plot, 'inflation_plot.png')

## Foreign FX Return (Indexed) ???

## Import, Export

%GDP
(%PGDP, Indexed)
Export partners

In [ ]:
# http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/BOP/Q.KZ.BXGS_BP6_USD+BMGS_BP6_USD?startPeriod=2015-01-01&endPeriod=2018-12-31
        

In [ ]:
trade_plot

In [ ]:
# Export 

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/BOP/Q.KZ.BXGS_BP6_USD' # adjust codes here
r = requests.get(f'{url}{key}').json()
obs = r['CompactData']['DataSet']['Series']['Obs']
Export = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
            float(i['@OBS_VALUE']) 
            for i in obs})

In [ ]:
# Import

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/BOP/Q.KZ.BMGS_BP6_USD' # adjust codes here
r = requests.get(f'{url}{key}').json()
obs = r['CompactData']['DataSet']['Series']['Obs']
Import = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
            float(i['@OBS_VALUE']) 
            for i in obs})

In [ ]:
trade = pd.DataFrame({'Export': Export, 'Import': Import}) #.to_csv('import_export.csv')

In [ ]:
#trade = pd.read_csv('import_export.csv', index_col=0, parse_dates=True)

In [ ]:
trade_pc = trade.div(debts1.iloc[:, 18], axis=0).div(1000).mul(100)

In [ ]:
trade_pc = trade_pc[((trade_pc.index> start) & (trade_pc.index < finish))]

In [ ]:
trade_pc['Export'].index

In [ ]:
#trade_pc.reset_index()

In [ ]:
%%opts Curve [width=900, height=250]
trade_plot = hv.Curve(trade_pc.reset_index(), 'index',
                         'Export', label='Export').\
                    options(title_format="Trade (%GDP)").opts(xlim=(start, finish),
                                                                ylabel='%', ylim=(0,20),
                                                              xlabel='Date',
                                                               padding=(0, (0, 0.5)))  *\
    arrow(trade_pc['Export'], label='Currency\ncrisis 14') * \
    hv.Curve(trade_pc.reset_index(), 'index',
                         'Import', label='Import')
trade_plot

In [ ]:
hv.save(trade_plot, 'trade_plot.png')

## Chg in Debt-GDP Ratio (Ann)
## Debt Growth (%GDP, Ann)

## Money1 Level (%GDP)

The idea is to plot two graphs : M!, M2, M3 in local cur, and the same in USD

In [ ]:
# Function

def get_imf_data(db='IFS', freq='Q', country='KZ', ind='FM0_SA_XDC'):
        url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
        #key = 'CompactData/BOP/Q.KZ.BK_DB_BP6_USD' # adjust codes here
        key = 'CompactData/' + db + '/' + freq + '.' + country + '.' + ind
        try:
            r = requests.get(f'{url}{key}').json()
        except:
            print('Something goes wrong')
            return None
        
        data = pd.DataFrame()
        try:
            obs = r['CompactData']['DataSet']['Series']['Obs']
            data = pd.Series({pd.to_datetime(i['@TIME_PERIOD']): 
                float(i['@OBS_VALUE']) 
                for i in obs})
        except:
            print('Smth goes wrong')
            
        return data

In [ ]:
money0 = get_imf_data(freq='M', ind='FM1_XDC')

In [ ]:
money0.to_csv('money1.csv', header=True)

In [ ]:
money_plot

In [ ]:
money1 = pd.read_csv('money1.csv', index_col=0, skiprows=1, parse_dates=True, names=['Money1'])

In [ ]:
money1 = money0
money1.name = 'Money1'

In [ ]:
money1.head()

In [ ]:
%%opts Curve [width=900, height=250]
money_plot = hv.Curve(money1.div(1e6).reset_index(), 'index',
                         'Money1', label='M1').\
                    options(title_format="Money 1").opts(xlim=(start, finish),
                                                         xlabel='Date',
                                                                ylabel='mln. KZT',
                                                               padding=(0, (0, 0.5))) *\
    arrow(money1.div(1e6), label='Currency\ncrisis 14') *\
    arrow(money1.div(1e6), date=crisis15, label='Currency\ncrisis 15') * \
    arrow(money1.div(1e6), date=crisis09, label='Currency\ncrisis 09')

money_plot

In [ ]:
hv.save(money_plot, 'money1_plot.png')

## Fiscal Balance (% GDP)
government budget balance


Net lending - borrowing (gross?)

## Nominal Long Rate
## Nominal Growth

## Real Short Rate

## Nominal Long Rate
## Nominal Short Rate

## Yield Curve (SR-LR)

## NBK Intervention

In [ ]:
interventions = pd.read_excel('nbk_data/interventions.xlsx')

In [ ]:
interventions.tail()

In [ ]:
int_nbk = interventions.melt(id_vars=['Reporting period'])
int_nbk.columns = ['Year', 'Month', 'Amount']
int_nbk.index = pd.to_datetime(int_nbk.apply(lambda x: str(x[0]) + '-' + x['Month'] + '-1', axis=1))

In [ ]:
int_nbk = int_nbk.sort_index().dropna() #
int_nbk.drop(columns = ['Year', 'Month'], inplace=True)
int_nbk.tail()

In [ ]:
int_nbk.iloc[int_nbk.index.get_loc(pd.to_datetime('2016-09-01'), method='nearest')].Amount

In [ ]:
#%%opts Curve [width=900, height=250]

inter_plot = hv.Curve(int_nbk.reset_index(), ('index', 'Time'),
                         'Amount', label='NBK Intervention').\
                    opts(title_format="NBK Intervention", ylabel='mln. USD', padding=(0, 0.5)) *\
    hv.HLine(0).opts(line_width=0.5, color='black') *\
    hv.Arrow(pd.to_datetime('2016-09-01'),
             int_nbk.iloc[int_nbk.index.get_loc(pd.to_datetime('2016-09-01'), method='nearest')].Amount,
             '"Let It Go"', 'v', arrowstyle='->') *\
    arrow(int_nbk, crisis14, label='Currency\ncrisis 14') *\
    arrow(int_nbk, crisis15, label='Currency\ncrisis 15')


In [ ]:
(currency_plot1 + inter_plot).cols(1)

In [ ]:
hv.save(inter_plot, 'inter_plot.png')

# Data search

### Dataflow Method

In [ ]:
# details: 
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  # Method with series information
search_term = ''  # Term to find in series names
series_list = requests.get(f'{url}{key}').json()\
            ['Structure']['Dataflows']['Dataflow']

In [ ]:
search_term = 'qqq'
# Use dict keys to navigate through results:
for series in series_list:
    if search_term in series['Name']['#text']:
        print(f"{series['Name']['#text']}: {series['KeyFamilyRef']['KeyFamilyID']}")

series code after ':', i.e' 'GFSR'

### DataStructure Method

In [ ]:
#CompactData/DOT/M.VN.TXG_FOB_USD.US+W00
key = 'DataStructure/IRFCL' #'DataStructure/GFSIBS'  # Method / series
dimension_list = requests.get(f'{url}{key}').json()\
            ['Structure']['KeyFamilies']['KeyFamily']\
            ['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f"Dimension {n}: {dimension['@codelist']}")

### CodeList Method

In [ ]:
# Example: codes for third dimension, which is 2 in python
key = f"CodeList/{dimension_list[3]['@codelist']}"
code_list = requests.get(f'{url}{key}').json()\
        ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list:
    print(f"{code['Description']['#text']}: {code['@value']}")

In [ ]:
code_list

### MetadataStructure Method

In [ ]:
#CompactData/DOT/M.VN.TXG_FOB_USD.US+W00
key = 'MetadataStructure/FM' #'DataStructure/GFSIBS'  # Method / series
dimension_list = requests.get(f'{url}{key}').json()\
            ['Structure']['KeyFamilies']['KeyFamily']\
            ['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f"Dimension {n+1}: {dimension['@codelist']}")